# Another Example in GMPL Notebook

This example is based on the American Engine Co. example in "Deterministic Operations Research: Models and Methods" by David J. Rader (Wiley, 2010).

> American Engine Co. produces two engines, one for trucks and one for cars.
During the next 3 months, they anticipate the following demands for their engines:

<table>
    <tr>
        <td></td>
        <td>Month 1</td>
        <td>Month 2</td>
        <td>Month 3</td>
    </tr>
    <tr>
        <td>Truck engines</td>
        <td>400</td>
        <td>300</td>
        <td>500</td>
    </tr>
    <tr>
        <td>Car engines</td>
        <td>800</td>
        <td>500</td>
        <td>600</td>
    </tr>
</table>

> Each month's demand must be fully satisfied.
During each month, at most 1000 engines (combined) can be produced.
Each truck engine requires 10 hours of labor to produce and costs \$2000 in supplies,
while each car engine requires 8 hours of labor and costs \$1500 in supplies.
At most 9000 hours are available each month.
At the beginning of month 1, 100 truck engines and 200 car engines are inventory.
At the end of each month, a holding cost of \$150 per engine is assigned to any engine in inventory.
At the end of the third month, management wants to have at least 100 of each engine in inventory.
How can we meet monthly demand at a minimum cost?

## Sets and parameters

Let's start by identifying sets and parameters for this problem.

We have a set of engine types (truck, car), as well as a set of time periods (months).
Let's define these sets as:

$$
\begin{aligned}
E & = \text{set of engine types}\\
T & = \text{set of time periods (months)}
\end{aligned}
$$

In GMPL, we can declare these sets like this:

In [ ]:
# Sets
set E;    # set of engine types
set T;    # set of time periods (months)

Next, let's define some symbols to represent the parameters given in the problem:

$$
\begin{alignedat}{2}
    d_{e,t} & = \text{demand for type $e$ engines in month $t$} &\quad& \text{for } e \in E, t \in T\\
    a_{e} & = \text{labor required to produce one type $e$ engine} &\quad& \text{for } e \in E\\
    c_{e} & = \text{cost of producing one type $e$ engine} &\quad& \text{for } e \in E\\
    I_{e,0} & = \text{initial inventory of type $e$ engines} &\quad& \text{for } e \in E
\end{alignedat}
$$

We declare these parameters in GMPL as follows:

In [ ]:
# Parameters
param d{e in E, t in T};    # demand for type e engines in month t
param a{e in E};            # labor required to produce one type e engine
param c{e in E};            # cost of producing one type e engine
param I0{e in E};           # initial inventory of type e engines

## Decision variables

Next, let's define decision variables. 

We want to determine how much of each engine is produced and how many engines are placed in inventory:

$$
\begin{alignedat}{2}
    x_{e, t} & = \text{number of type $e$ engines to produce in month $t$} &\quad& \text{for } e \in E, t \in T\\
    y_{e, t} & = \text{number of type $e$ engines in inventory at the end of month $t$} &\quad& \text{for } e \in E, t \in T \cup \{0\}
\end{alignedat}
$$

We declare these decision variables in GMPL like this:

In [ ]:
# Decision variables
var x{e in E, t in T} >= 0;             # number of type e engines to produce in month t
var y{e in E, t in T union {0}} >= 0;   # number of type e engines in inventory at the end of month t

Note that we can impose bounds on the variable (in this case, $\ge 0$) when we declare the variables.

## Objective function

The objective here is to minimize costs. There are two components to the cost: holding costs and supply costs.
The holding costs can be expressed as

$$
\text{holding costs} = 150 \sum_{e \in E} \sum_{t \in T} y_{e,t}
$$

The supply costs are

$$
\text{supply costs} = \sum_{e \in E} c_e \sum_{t \in T} x_{e,t}
$$

Adding these together, we obtain our objective function. We can write this in GMPL as follows:

In [ ]:
# Objective function
maximize total_cost: 150 * sum{t in T, e in E} y[e,t] + sum{e in E, t in T} c[e] * x[e,t];

## Constraints

First, we have constraints on the number of engines produced in each month:

$$
\sum_{e \in E} x_{e,t} \le 1000 \quad \text{for } t \in T
$$

These constraints can be expressed in GMPL as:

In [ ]:
# Constraints
subject to engines_per_month{t in T}:
    sum{e in E} x[e,t] <= 1000;

We also have constraints on the amount of labor used each month:

$$
\sum_{e \in E} a_{e} x_{e,t} \le 9000 \quad \text{for } t \in T
$$

These constraints can be written in GMPL as:

In [ ]:
subject to labor_per_month{t in T}:
    sum{e in E} a[e] * x[e,t] <= 9000;

We also need to make sure that the inventory levels, production levels, and demand are compatible with each other. In particular, we need to ensure that

$$
\left( \begin{array}{c}
    \text{Inventory at the}\\
    \text{end of month $t-1$}
\end{array} \right) 
+
\left( \begin{array}{c}
    \text{Production}\\
    \text{during month $t$}
\end{array} \right)
=
\left( \begin{array}{c}
    \text{Demand}\\
    \text{during month $t$}
\end{array} \right) 
+
\left( \begin{array}{c}
    \text{Inventory at the}\\
    \text{end of month $t$}
\end{array} \right)
$$

Therefore, we need the following **balance constraints**:

$$
y_{e,t-1} + x_{e,t} = d_{e,t} + y_{e,t} \quad \text{for } e \in E, t \in T
$$

In GMPL:

In [ ]:
subject to balance{e in E, t in T}:
    y[e,t-1] + x[e,t] = d[e,t] + y[e,t];

Finally, we have some constraints on the initial and ending inventory levels:

$$
\begin{alignedat}{2}
    y_{e,0} & = I_{e,0} &\quad& \text{for } e \in E\\
    y_{e,3} & \ge 100 &\quad& \text{for } e \in E
\end{alignedat}
$$

We can write these constraints in GMPL like this:

In [ ]:
subject to initial_inventory{e in E}:
    y[e,0] = I0[e];
    
subject to ending_inventory{e in E}:
    y[e,3] >= 100;

## Data

It's now time to populate the sets and parameters with actual values! This is accomplished in the `data` section of a GMPL model file.

In [ ]:
data;

We define the sets $E$ and $T$ in GMPL as follows:

In [ ]:
# Sets
set E := "truck" "car";
set T := 1 2 3;

Next, we can give values for the parameters $d_{e,t}$, $a_{e}$, $c_{e}$ and $I_{e,0}$ in GMPL like this:

In [ ]:
# Parameters
param d :
        1   2   3 :=
truck 400 300 500
car   800 500 600;

param a :=
    truck   10
    car     8;
    
param c :=
    truck   2000
    car     1500;

param I0 :=
    truck   100
    car     200;

## Finishing up

We end our GMPL file with an end statement:

In [ ]:
end;

To solve this model in GMPL notebook, press the <i class="fa fa-calculator" aria-hidden="true"></i> button in the toolbar. This will collate all the GMPL code we wrote above and solve it using GLPK.

In [ ]:
%solve